## IN3062 Introduction to AI Project
### Spotify Machine Learning Experiments

Being a fan of music and an avid user of Spotify, I decided to use a dataset of over 160,000 songs each with granular information behind their charataristics such as: acousticness, energy, danceability and tempo. 

## Predicting Song Popularity

Each song has a popularity score between 0 and 100 indicating how popular the song is with worldwide Spotify users, this interested me as I have the hypothesis: Is it possible to predict how popular a song could be based purely on characteristics?

Let's explore this hypothesis...

### Load The Dataset

The datasets can be located at https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks. 

I am using the main data.csv file to start, I will explore the additional datasets: data_by_genre.csv and data_by_artists later in the notebook.

In [1]:
# Imports all of the required libraries.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn import linear_model
import matplotlib.pyplot as plt

# Sets a new DataFrame to the contents of 'data.csv'

df = pd.read_csv('data.csv')


### Exploring The Dataset

Taking a look at the columns of the dataset we can see that each song has:

'valence' = The positiveness of the song.

'year' = The release year of the song.

'acousticness' = The relative metric of the song being acoustic (not having electrical amplification).

'artists' = An array containing the artist(s) name's.

'danceability' = The relative measurement of the song being danceable.

'duration_ms' = The duration of the song in miliseconds.

'energy' = The relative measurement of the song's energy.

'explicit' = A Boolean value indicating if the song contains explicit lyrics.

'id' = The inique identifer for the song.

'instrumentalness' = The relative ratio of the song being instrumental.

'key' = The musical key that the song is recorded in. Represented as Integers between 0 and 11.

'liveness' = The relative duration of the song sounding like a live performance.

'loudness' = The relative loudness of the song in the range of [-60, 0] decibels (dB).

'mode' = A Boolean value indicating if the song starts with a major chord progression or not.

'name' = The name of the song.

'popularity' = The current popularity of the song measured by Spotify. Represented as Integers between 0 and 100.

'release_date' = The date of release of the song in yyyy-mm-dd, yyyy-mm or yyyy format.

'speechiness' = The relative length of the song containing any kind of vocals.

'tempo' = The musical tempo (speed) of the song.


In [2]:
df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


Let's see what the top five songs are according to popularity, to judge if there is a trend in their characteristics.

In [3]:
df.nlargest(5, 'popularity')

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
19611,0.145,2020,0.4010,"['Bad Bunny', 'Jhay Cortez']",0.731,205090,0.573,1,47EiUVwUp4C9fGccaPuUCS,0.000052,4,0.1130,-10.059,0,Dakiti,100,2020-10-30,0.0544,109.928
19606,0.756,2020,0.2210,"['24kGoldn', 'iann dior']",0.700,140526,0.722,1,3tjFYV6RSFtuktYl3ZtYcq,0.000000,7,0.2720,-3.558,0,Mood (feat. iann dior),99,2020-07-24,0.0369,90.989
19618,0.737,2020,0.0112,['BTS'],0.746,199054,0.765,0,0t1kP63rueHleOhQkYSXFY,0.000000,6,0.0936,-4.410,0,Dynamite,97,2020-08-28,0.0993,114.044
19608,0.357,2020,0.0194,"['Cardi B', 'Megan Thee Stallion']",0.935,187541,0.454,1,4Oun2ylbjFKMPTiaSbbCih,0.000000,1,0.0824,-7.509,1,WAP (feat. Megan Thee Stallion),96,2020-08-07,0.3750,133.073
19610,0.682,2020,0.4680,['Ariana Grande'],0.737,172325,0.802,1,35mvY5S1H3J2QZyna3TFe0,0.000000,0,0.0931,-4.771,1,positions,96,2020-10-30,0.0878,144.015


Apart from not enjoying any of these songs myself, we can draw that the main similarities that they exhibit are that they all have no or very low instrumentalness, a danceability > 7.0 and they were all released in 2020.

So as a generalisation, we can say that 'dancy' songs released in 2020 with no instrumentals are likley to be popular. Lets see if the training can pick up on this later.

### Preparing The Dataset

If we take a look at the info behind the data frame, we can see that there are no null values for each column which is great as we won't need to remove any null values which would have resulted in losing some data.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

We can also see that the data types for the majority of the columns are either Integer or Float values which are ready for training. However for 'artists', 'id', 'name' and 'release_date'. I will touch on 'artists' and 'name' later when I attempt to increase the accuracy of the predictions but for now we can drop it from the dataframe along with: 'id' and 'release_date'.

In [5]:
baselineDf = df.drop(columns=['id', 'artists', 'release_date', 'name', 'mode']).copy()

### Baseline Training

Now that the DataFrame has had the non-numeric objects removed we can start with the baseline training.
To start I am going to create two dataframes, one for the dataIn (X) and one for the dataOut (y). 

Then using Scikit-learn's test_train_spilt I am going to split the training and testing data 80:20. 

And finally I will run the training data though a Linear Regression model, then test the predictions against the test data to get an accuracy score.

The accuracy score is derived from the following measures:

Mean Squared Error (MSE) - The mean average of which predicted data points stray away from the regresssion line. Lower is better.

R Squared (R^2) - The statistical measure of how close the predicted data points are to the fitted regression line.
The closer to 1 the better.

With the R^2 score being multiplied by 100 and represented to five significant figures as the overall accuracy score.

In [6]:
# Defines a function that takes a given DataFrame, splits the data into testing and training sets then applies Linear Regression.

def train(data):
    
    # Sets the subject and target DataFrames. 
    
    dataIn = data.drop(columns=['popularity'])
    dataOut = data['popularity']
    
    # Splits the training and test data 80:20 and ensures that with each test it doesn't change.

    dataInTrain, dataInTest, dataOutTrain, dataOutTest = train_test_split(dataIn, dataOut, test_size=0.2, random_state=0)

    # Defines the LinearRegression model and applies the input and output data to the model for training.
    
    model = linear_model.LinearRegression()
    model.fit(dataInTrain, dataOutTrain)

    # Uses the test data and trained model to make predictions.
    
    predictions = model.predict(dataInTest)
    
    # Calculates and outputs the accuracy score.
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    
    return accuracy

print("=== Baseline Accuracy: {:.5f}% ===".format((train(baselineDf)) * 100))


MSE: 113.74587287017013
R^2: 0.7611216828203751
Accuracy: 76.11217%

=== Baseline Accuracy: 76.11217% ===


Just from this initial basline we can see that the accuracy is 76.11217% accurate at predicting the popularity of a song based on it's characteristics.

Let's see if we can improve this by adjusting the dataset.

### Adjusting The Dataset

The first thing I want to try is to add artists to the training model. As some artists may be so popular that it is likley that when they release a song it is going to have a high popularity score.

To achieve this I have decided to use a LabelEncoder to convert the arrays of artists names to a unique Integer value so it can be trained in the model alongside the rest of the data.

In [7]:
# Imports sklearn's LabelEncoder library.

from sklearn.preprocessing import LabelEncoder

# Creates a new DataFrame for the adjustments.

adjustmentDf = df.drop(columns=['id', 'release_date', 'name', 'mode']).copy()

# A function to apply the artists adjustment to a DataFrame.

def encodeStrings(data, column):

    le = LabelEncoder()
    return le.fit_transform(data[column].astype('str'))

# Applies function to the artists column.

adjustmentDf['artists'] = encodeStrings(adjustmentDf, 'artists')

# Trains the new adjustment DataFrame and outputs the results.

print("=== Artist Adjustment Accuracy: {:.5f}% ===".format((train(adjustmentDf)) * 100))


MSE: 113.67924490171757
R^2: 0.7612616085739821
Accuracy: 76.12616%

=== Artist Adjustment Accuracy: 76.12616% ===


With this small improvement we can see a gain of 0.01399% in accuracy. Not the greatest improvement, but improvement nonetheless.

Next I want to modify the dataframe even further, this time by applying a function to some of the columns to generalise the data to make matches more likley.

I am going to define two functions:

normaliseMetrics : Rounds the given column's value's to two decimal places.

normaliseTempo: Rounds the tempo column's value's to the nearest ten.


In [8]:
# Function to round a given column's value's to two decimal places.

def normaliseMetrics(value):
    return round(value, 2)

# Function to round the tempo column's value's to the nearest ten.

def normaliseTempo(value):
    return round(abs(value), -1)


I will now apply these modifier functions to the relevent columns in the dataframe.

In [9]:
# Applies the normaliseMetrics function to each of the following columns.

adjustmentDf['valence'] = adjustmentDf['valence'].apply(normaliseMetrics)
adjustmentDf['acousticness'] = adjustmentDf['acousticness'].apply(normaliseMetrics)
adjustmentDf['danceability'] = adjustmentDf['danceability'].apply(normaliseMetrics)
adjustmentDf['energy'] = adjustmentDf['energy'].apply(normaliseMetrics)
adjustmentDf['instrumentalness'] = adjustmentDf['instrumentalness'].apply(normaliseMetrics)
adjustmentDf['liveness'] = adjustmentDf['liveness'].apply(normaliseMetrics)
adjustmentDf['loudness'] = adjustmentDf['loudness'].apply(normaliseMetrics)
adjustmentDf['speechiness'] = adjustmentDf['speechiness'].apply(normaliseMetrics)

# Applies the normaliseMetrics function to the tempo column.

adjustmentDf['tempo'] = adjustmentDf['tempo'].apply(normaliseTempo)

adjustmentDf.head(5)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,popularity,speechiness,tempo
0,0.06,1921,0.98,26839,0.28,831667,0.21,0,0.88,10,0.67,-20.10,4,0.04,80.0
1,0.96,1921,0.73,7382,0.82,180533,0.34,0,0.00,7,0.16,-12.44,5,0.41,60.0
2,0.04,1921,0.96,16378,0.33,500062,0.17,0,0.91,3,0.10,-14.85,5,0.03,110.0
3,0.17,1921,0.97,10077,0.28,210000,0.31,0,0.00,5,0.38,-9.32,3,0.04,100.0
4,0.25,1921,0.96,23719,0.42,166693,0.19,0,0.00,3,0.23,-10.10,2,0.04,100.0


As you can see the dataframe has now had each of the relevent columns modified to make the data more general.

Now we can run the updated dataframe though the training model.

In [10]:
print("=== Adjustment Average Accuracy: {:.5f}% ===".format((train(adjustmentDf)) * 100))

MSE: 113.68077577361474
R^2: 0.7612583935817069
Accuracy: 76.12584%

=== Adjustment Average Accuracy: 76.12584% ===


As it turns out this adjustment has actually decreased the accuracy by -0.00032%, proving that the regression works as effectivly even if the data is more general.

### Adding Genres

Next up I will now use the 'data_w_genres.csv' dataset to provide each song in the existing data.csv dataset with the genres associated with each accredited artist. 

If we take a look at: Lana Del Rey, Royal Blood and Bill Withers as example artists, we can see their associated genres.

In [11]:
# Loads the data_w_genres.csv as a new DataFrame.

genresDf = pd.read_csv('data_w_genres.csv')

genresDf.loc[genresDf['artists'] ==  "Lana Del Rey"]

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
14100,"['art pop', 'pop']",Lana Del Rey,0.44669,0.423221,256964.019608,0.542764,0.020926,0.17142,-8.448186,0.055098,115.393554,0.254493,57.892157,0,0,204


In [12]:
genresDf.loc[genresDf['artists'] ==  "Royal Blood"]

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
21432,"['brighton indie', 'garage rock', 'modern alte...",Royal Blood,0.04634,0.5242,212152.8,0.876,0.00208,0.15664,-4.4422,0.0574,107.982,0.5894,54.6,0,1,10


In [13]:
genresDf.loc[genresDf['artists'] ==  "Bill Withers"]

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
2701,"['funk', 'motown', 'quiet storm', 'soul']",Bill Withers,0.471254,0.653619,257137.321429,0.468529,0.065161,0.209877,-12.269083,0.064256,111.738393,0.629061,40.52381,9,1,168


The function below will iterate over each song in the data.csv dataset and for each of the accredited artists will lookup in the data_w_genres.csv dataset what the genres are for the given artist. Then export the new dataset as a csv called: songs_w_genres.csv

#### Warning: 

This process requires a lot of data processing and will be an intensive task on your computer. It took my 2020 MacBook Pro with a 2.3 GHz Quad-Core Intel Core i7 and 16 GB of Memory over 25 minutes at 100% CPU to process this. So if you don't wish to run this I have provided an already processed songs_w_genres.csv in the repository.

In [14]:
def generateSongsWithGenres():
    
    # Creates a new DataFrame based on the existing data.csv.

    genreAdjustmentDf = df.copy()

    # Adds a 'genres' column to the new DataFrame.

    genreAdjustmentDf['genres'] = ''

    # Iterates over every row in the new DataFrame.

    for index, row in genreAdjustmentDf.iterrows():
    
        # Parses the string data in the 'artists' column into a list of individual artists that are acredited to the given song. 
    
        artists = row.get('artists').replace('[', '').replace(']', '').replace("'", '').split(', ')
    
        genresList = []
    
        # Iterates over each artist in the accredited list.
    
        for artist in artists:
        
            # Searches the artists with genres DataFrame for a match with the given artist. Returns the row as a Series.
        
            target = genresDf.loc[genresDf['artists'] == artist].squeeze()
        
            # Parses the string data in the 'genres' column into a list of individual genres that are assosiated with the given artist. Appends this list to an existing list if there are multiple given artists.
        
            genresList = genresList + str(target.get('genres')).replace('[', '').replace(']', '').replace("'", '').split(', ')
        
        # Removes any duplicate genres in the list and appends it as a string to the new genres column for the given song.

        genreAdjustmentDf.loc[index, 'genres'] = str(list(set(genresList)))
    
        print('Parsed ' + str(index + 1) + ' of ' + str(len(genreAdjustmentDf.index)))
    
    # Exports the new DataFrame as a new csv file called 'songs_w_genres.csv' so it can be used without needing to parse again.
                         
    genreAdjustmentDf.to_csv(r'songs_w_genres.csv', index = False)


#Uncomment the line below if you wish to generate the 'songs_w_genres.csv' file.

#generateSongsWithGenres()
        

If we now look up a songs by the previously given artists we can see that on the end there is now a 'genres' column at the end of the DataFrame which features the genres of the accredited artists.

In [15]:
# Sets a new DataFrame to the dataset from 'songs_w_genres.csv'.

songsGenresDf = pd.read_csv('songs_w_genres.csv')

# Obtains the row for the given song.

songsGenresDf.loc[songsGenresDf['name'] == 'Summertime Sadness'].head(1)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,genres
18137,0.235,2012,0.0542,['Lana Del Rey'],0.565,265427,0.654,0,2dBwB667LHQkLhdYlwLUZK,0.000002,1,0.122,-6.826,0,Summertime Sadness,63,2012-11-12,0.0335,111.968,"['art pop', 'pop']"


In [16]:
songsGenresDf.loc[songsGenresDf['name'] == 'Out of the Black'].head(1)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,genres
74002,0.633,2014,0.000867,['Royal Blood'],0.52,240572,0.981,1,2nIulsLMiP2SMTDrtxfQXX,0.00177,2,0.197,-2.75,1,Out of the Black,53,2014-08-25,0.0906,158.937,"['modern blues rock', 'garage rock', 'modern a..."


In [17]:
songsGenresDf.loc[songsGenresDf['name'] == 'Lovely Day'].head(1)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,genres
11013,0.706,1977,0.292,['Bill Withers'],0.69,254560,0.651,0,0bRXwKfigvpKZUurwqAlEh,0.00241,9,0.105,-8.267,1,Lovely Day,76,1977-10-29,0.0324,97.918,"['motown', 'funk', 'quiet storm', 'soul']"


Also if we take a look at a song that features multiple accredited artists we can see that the genres of both artists have been combined, without duplicating genres.

In [18]:
# Gets the details of a given song as a Series.

target = songsGenresDf.loc[songsGenresDf['name'] == 'Endless Love'].head(1).squeeze()

# Outputs the artists of the song.

print('Artists: ' + str(target.get('artists')) + '\n')

# Outputs the genres associated with each artist.

print('Lionel Richie - Genres: ' + str(genresDf.loc[genresDf['artists'] == 'Lionel Richie'].squeeze().get('genres')) + '\n') 
print('Diana Ross - Genres: ' + str(genresDf.loc[genresDf['artists'] == 'Diana Ross'].squeeze().get('genres')) + '\n')

# Outputs the combined genres from each artist without duplicates.

print('Combined Genres for Endless Love: ' + str(target.get('genres')))

Artists: ['Lionel Richie', 'Diana Ross']

Lionel Richie - Genres: ['adult standards', 'disco', 'mellow gold', 'motown', 'quiet storm', 'soft rock']

Diana Ross - Genres: ['adult standards', 'disco', 'funk', 'mellow gold', 'motown', 'quiet storm', 'soft rock', 'soul']

Combined Genres for Endless Love: ['soft rock', 'funk', 'quiet storm', 'motown', 'mellow gold', 'disco', 'adult standards', 'soul']


### Testing Genres

So now that we have the genres featured for each song, let's see if this will improve the accuracy of the Linear Regression model.

Firstly we need to drop the irrelevant columns from the new data frame. Then we need to encode the genres into unique integer values the same way we did with the artists earlier. So we can use the same function: 'encodeStrings()'


In [19]:
# Sets a new DataFrame to the dataset from 'songs_w_genres.csv'.

songsWithGenresDf = pd.read_csv('songs_w_genres.csv')

# Removes unnecessary columns.

songsWithGenresDf = songsWithGenresDf.drop(columns=['id', 'release_date', 'name', 'mode'])

# Applies the 'encodeStrings()' function so the string data can be ready for training.

songsWithGenresDf['artists'] = encodeStrings(songsWithGenresDf, 'artists')
songsWithGenresDf['genres'] = encodeStrings(songsWithGenresDf, 'genres')

# Runs the training and outputs the results.

print("=== Genre Addition Accuracy: {:.5f}% ===".format((train(songsWithGenresDf)) * 100))

MSE: 113.38291980221359
R^2: 0.7618839225034588
Accuracy: 76.18839%

=== Genre Addition Accuracy: 76.18839% ===


This has added 0.06223% to the accuracy a massive improvement.

To check if Linear Regression is the best Regressor for this task I will try RidgeCV regressor 

In [20]:
def extraTrain(data):
    
    # Sets the subject and target DataFrames. 
    
    dataIn = data.drop(columns=['popularity'])
    dataOut = data['popularity']

    dataInTrain, dataInTest, dataOutTrain, dataOutTest = train_test_split(dataIn, dataOut, test_size=0.2, random_state=0)

    # Tests Ridge Regressor
    
    model = linear_model.Ridge()
    model.fit(dataInTrain, dataOutTrain)

    predictions = model.predict(dataInTest)
    
    print("Ridge Regressor")
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    
    # Tests SGDRegressor Regressor
    
    model = linear_model.SGDRegressor()
    model.fit(dataInTrain, dataOutTrain)

    predictions = model.predict(dataInTest)
    
    print("SGDRegressor Regressor")
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    
    # Tests ElasticNet Regressor
    
    model = linear_model.ElasticNet()
    model.fit(dataInTrain, dataOutTrain)

    predictions = model.predict(dataInTest)
    
    print("ElasticNet Regressor")
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    
    # Tests Lars Regressor
    
    model = linear_model.Lars()
    model.fit(dataInTrain, dataOutTrain)

    predictions = model.predict(dataInTest)
    
    print("Lars Regressor")
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    
    # Tests Lasso Regressor
    
    model = linear_model.Lasso()
    model.fit(dataInTrain, dataOutTrain)

    predictions = model.predict(dataInTest)
    
    print("Lasso Regressor")
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    
    # Tests LassoLars Regressor
    
    model = linear_model.LassoLars()
    model.fit(dataInTrain, dataOutTrain)

    predictions = model.predict(dataInTest)
    
    print("LassoLars Regressor")
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    
    # Tests HuberRegressor Regressor
    
    model = linear_model.HuberRegressor()
    model.fit(dataInTrain, dataOutTrain)

    predictions = model.predict(dataInTest)
    
    print("HuberRegressor Regressor")
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    
    # Tests OrthogonalMatchingPursuit Regressor
    
    model = linear_model.OrthogonalMatchingPursuit()
    model.fit(dataInTrain, dataOutTrain)

    predictions = model.predict(dataInTest)
    
    print("OrthogonalMatchingPursuit Regressor")
    
    print("MSE: " + str(mean_squared_error(dataOutTest, predictions)))

    print("R^2: " + str(r2_score(dataOutTest, predictions)))

    accuracy = model.score(dataInTest, dataOutTest)
    
    print("Accuracy: {:.5f}%\n".format(accuracy * 100))
    

extraTrain(songsWithGenresDf)


Ridge Regressor
MSE: 113.38286109455854
R^2: 0.7618840457957198
Accuracy: 76.18840%

SGDRegressor Regressor
MSE: 8.388857371030632e+37
R^2: -1.7617484320850615e+35
Accuracy: -17617484320850614756165834615006691328.00000%

ElasticNet Regressor
MSE: 116.71208726813478
R^2: 0.7548923200672444
Accuracy: 75.48923%

Lars Regressor
MSE: 113.38291980221594
R^2: 0.7618839225034539
Accuracy: 76.18839%

Lasso Regressor
MSE: 116.72797734765051
R^2: 0.7548589492260984
Accuracy: 75.48589%

LassoLars Regressor
MSE: 476.2306201649208
R^2: -0.0001344775317937419
Accuracy: -0.01345%

HuberRegressor Regressor
MSE: 443.00548605333785
R^2: 0.06964180466968306
Accuracy: 6.96418%

OrthogonalMatchingPursuit Regressor
MSE: 117.94334271441588
R^2: 0.7523065539062024
Accuracy: 75.23066%



As you can see from the results there are no clear winners with only: Ridge and Lars Regressors coming close to the results with standard LinearRegression.

## Artist and Song Recommendations

Something that has always interested me about Spotify is how it's able to recommend artists and songs to it's users.

It does this using two recomendation methods:

#### Content Based Filtering

Content Based Filtering works by taking the data attributes that are already known about a given artist or song, then matches similar artists and songs based on calculating the similarities between these attributes.

#### Collaborative Filtering

Collaberative Filtering works by taking users listening or rating history of songs and artists, then building recomendations based on this history.

### Artist Recommendations

Seeing as there are no records of listening history I cannot devise a Collaborative Filtering method for recomendations, so I have devised a Content Based Filtering recomendations system that takes a given artist then will recommend similar artists based on the genres that the given artist are known for.

It is fundamentally based on converting the 'genres' string into a matrix of token counts using SciKitLearn's CountVectorizer. This matrix is then passed into SciKitLearn's cosine_similarity() function which applies mathematical Cosine Similarity Theorem to generate a similarity score of between 1 and 0 for each individual combination of artists. So say if there are 1000 artists there would be 1000 x 1000 combinations calculated.

The given artist then has their similarities measured and then the top ten highest ratings are extracted and displayed with relevent information alongside.

In [21]:
# Importing the required libraries for vectorising and applying Cosine Similarity.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sets a new DataFrame based on 'data_w_genres.csv'.

recomendationDF = pd.read_csv('data_w_genres.csv')

# Removes unnecessary columns.

recomendationDF = recomendationDF.drop(columns=['count', 'mode', 'valence','tempo','loudness','acousticness','danceability','duration_ms','energy','instrumentalness','key','liveness','speechiness'])

# Function for recommending artists based on a given DataFrame and artist name. 

def recommendArtists(data, artistName):
    
    print('Processing recomendations for ' + str(artistName) + '...')
    
    # Obtains the index of the given artist.
    
    target = data.loc[data['artists'] == artistName].head(1).index.item()
    
    #Converts 'genres' string into a matrix of token counts. Then applies Cosine Similarity to the matrix.
    
    vectoriser = CountVectorizer(stop_words = 'english')
    countMatrix = vectoriser.fit_transform(data['genres'])
    cosineSimMatrix = cosine_similarity(countMatrix, countMatrix)
    
    # Derives similarity score based on provided artist, and sorts results in decending order.
    
    similarityScore = pd.Series(cosineSimMatrix[target]).sort_values(ascending = False)
    
    # Defines new DataFrame for results.
    
    rec = similarityScore.iloc[1:11].to_frame(name = 'similarity')
        
    rec['artist'] = ''
    rec['genres'] = ''
    
    # Appends artist information to the output DataFrame.
    
    for index, row in rec.iterrows():
        
        rec.loc[index, 'artist'] = str(list(data['artists'])[index])
        rec.loc[index, 'genres'] = str(list(data['genres'])[index])
        
    print('Results:')
                                                 
    return rec

recommendArtists(recomendationDF, 'Arctic Monkeys')


Processing recomendations for Arctic Monkeys...
Results:


,similarity,artist,genres
25627,0.901498,The Strokes,"['alternative rock', 'garage rock', 'modern ro..."
17639,0.878310,Muse,"['modern rock', 'permanent wave', 'rock']"
735,0.878310,Alex Turner,"['modern rock', 'rock', 'sheffield indie']"
25042,0.878310,The Killers,"['modern rock', 'permanent wave', 'rock']"
28120,0.843274,alt-J,"['indie rock', 'modern rock']"
25256,0.843274,The National,"['indie rock', 'modern rock']"
13189,0.841334,Kasabian,"['alternative rock', 'britpop', 'garage rock',..."
25414,0.840168,The Raconteurs,"['alternative rock', 'garage rock', 'garage ro..."
2853,0.839061,Black Rebel Motorcycle Club,"['alternative rock', 'blues rock', 'dance-punk..."
17505,0.839061,Motorcycle,"['alternative rock', 'blues rock', 'dance-punk..."


As you can see on the output DataFrame each of the recommended artists has a 'similarity' rating which indicates how close the algortithm determined the other artists to be.

I'm actually rather impressed as the algorithm ranked 'Alex Turner' highly and being the frontman of the Arctic Monkeys is a testament to it's performance.

Try it out for yourself, replace the 'The Who' string below with another artist and it should provide recomendations.

In [22]:
recommendArtists(recomendationDF, 'The Who')

Processing recomendations for The Who...
Results:


,similarity,artist,genres
25051,0.976606,The Kinks,"['album rock', 'art rock', 'british invasion',..."
9543,0.967629,Grand Funk Railroad,"['album rock', 'art rock', 'blues rock', 'clas..."
17425,0.965609,Mood,"['album rock', 'art rock', 'british invasion',..."
25224,0.965609,The Moody Blues,"['album rock', 'art rock', 'british invasion',..."
11291,0.964633,James Gang,"['album rock', 'art rock', 'blues rock', 'brit..."
23279,0.962514,Steely Dan,"['album rock', 'art rock', 'classic rock', 'co..."
5156,0.961557,Cream,"['album rock', 'art rock', 'blues rock', 'clas..."
26137,0.961202,Tom Petty and the Heartbreakers,"['album rock', 'classic rock', 'country rock',..."
23416,0.961202,Steve Miller Band,"['album rock', 'blues rock', 'classic rock', '..."
23352,0.960443,Steppenwolf,"['acid rock', 'album rock', 'blues rock', 'cla..."


### Song Recommendations

Using the same Content Based Filtering for artists and the 'songs_w_genres.csv' dataset I generated, I can device a system for recommending songs based on the genres in the songs.

As you can see the implementation is identical, apart from: the function's signature, column names and dataset are different.

However, when developing this I ran into an unavoidable error when generating the cosine similarity matrix to measure the similarities. As the dataset contains over 170,000 songs whenever I would run the function the program would hang and eventually time out with an error stating: "nnz of the result is too large" when it tries to generate the Cosine Similarity matrix. After some research it turns out this error is rooted in memory overflow as the 170,000 x 170,000 matrix is too large for the memory to handle. So I had to resort to limiting the dataset to 30,000 entries which you can see by my implementation of .iloc[0:30000], which limits the dataset to 30,000 entries.

So currently if the user enters a song that has an index of > 30,000 the function will crash and not work.

I attempted to remedy this by employing a "moving window" approach for the dataset, so that when the user enters a song title eg: "Arabella" it would use it's index, in this case: 18332, and generate a higher and lower bound both +/- 15,000 respectively. This would create a window of search ranging from indexes: 3332 to 33332 for the Cosine Similarity matrix to be generated. However, I ran into problems as the matrix's index still ranged between 0 and 30000. So when it came to checking the song's index for comparisons, it was either the incorrect index or an out of bounds error. 

Unfortunate as it may be, it still works and below are some examples of songs that have indexes < 30000.

In [23]:
# Sets a new DataFrame based on 'songs_w_genres.csv'.

recomendationDF = pd.read_csv('songs_w_genres.csv')

# Removes unnecessary columns.

recomendationDF = recomendationDF.drop(columns=['id', 'release_date', 'mode', 'valence','tempo','loudness','acousticness','danceability','duration_ms','energy','explicit','instrumentalness','key','liveness','year','speechiness'])

# Function for recommending songs based on a given DataFrame and song name. 

def recommendSongs(data, songName):
    
    print('Processing recomendations for ' + str(songName) + '...')
    
    # Obtains the index of the given artist.
    
    target = data.loc[data['name'] == songName].head(1).index.item()
    
    #Converts 'genres' string into a matrix of token counts. Then applies Cosine Similarity to the matrix.
    
    vectoriser = CountVectorizer(stop_words = 'english')
    countMatrix = vectoriser.fit_transform(data['genres'].iloc[0:30000])
    cosineSimMatrix = cosine_similarity(countMatrix, countMatrix)
    
    # Derives similarity score based on provided song, and sorts results in decending order.
    
    similarityScore = pd.Series(cosineSimMatrix[target]).sort_values(ascending = False)
    
    # Defines new DataFrame for results.
    
    rec = similarityScore.iloc[1:19].to_frame(name = 'similarity')
        
    rec['name'] = ''
    rec['artists'] = ''
    rec['genres'] = ''
    
    # Appends song information to the output DataFrame.
    
    for index, row in rec.iterrows():
        
        rec.loc[index, 'name'] = str(list(data['name'])[index])
        rec.loc[index, 'artists'] = str(list(data['artists'])[index])
        rec.loc[index, 'genres'] = str(list(data['genres'])[index])
        
    print('Results:')
                                                 
    return rec

recommendSongs(recomendationDF, 'Arabella')


Processing recomendations for Arabella...
Results:


,similarity,name,artists,genres
16886,1.000000,When The Sun Goes Down,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
18405,1.000000,Snap Out Of It,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
16964,1.000000,Mardy Bum,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
17205,1.000000,Teddy Picker,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
16970,1.000000,I Bet You Look Good On The Dancefloor,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
17197,1.000000,Brianstorm,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
17189,1.000000,Old Yellow Bricks,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
18241,1.000000,I Wanna Be Yours,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
17006,1.000000,505,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."
16994,1.000000,Baby I'm Yours,['Arctic Monkeys'],"['sheffield indie', 'garage rock', 'rock', 'pe..."


In [24]:
recommendSongs(recomendationDF, 'Lovely Day')

Processing recomendations for Lovely Day...
Results:


,similarity,name,artists,genres
4091,1.0,Ain't No Sunshine,['Bill Withers'],"['motown', 'funk', 'quiet storm', 'soul']"
16012,1.0,Superstition - Single Version,['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"
10823,1.0,Sir Duke,['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"
29221,1.0,He's Misstra Know-It-All,['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"
29222,1.0,All In Love Is Fair,['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"
11797,1.0,Lately,['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"
10376,1.0,Visions,['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"
12171,1.0,Ribbon In The Sky,['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"
11786,1.0,Master Blaster (Jammin'),['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"
10387,1.0,Living For The City,['Stevie Wonder'],"['motown', 'funk', 'quiet storm', 'soul']"


In [25]:
recommendSongs(recomendationDF, 'Summertime Sadness')

Processing recomendations for Summertime Sadness...
Results:


,similarity,name,artists,genres
18126,1.000000,Video Games - Remastered,['Lana Del Rey'],"['art pop', 'pop']"
18137,1.000000,Summertime Sadness,['Lana Del Rey'],"['art pop', 'pop']"
18590,1.000000,Brooklyn Baby,['Lana Del Rey'],"['art pop', 'pop']"
18488,0.912871,Runaway,['AURORA'],"['norwegian pop', 'art pop']"
19288,0.894427,all the kids are depressed,['Jeremy Zucker'],['pop']
19461,0.894427,i love you,['Billie Eilish'],['pop']
19263,0.894427,Let Me Down Slowly,['Alec Benjamin'],['pop']
19433,0.894427,when the party's over,['Billie Eilish'],['pop']
19270,0.894427,comethru,['Jeremy Zucker'],['pop']
18743,0.894427,Dollhouse,['Melanie Martinez'],['pop']


As you can see the results for 'Arabella' by the Arctic Monkeys's are very similar to each other, mainly songs by the original artist which is expected as the genres remain consistant.

With 'Lovley Day' by Bill Withers the results are interesting as there are more songs by Stevie Wonder than Bill, this is due to his genres being exactly the same as Bills.

And finally with 'Summertime Sadness' by Lana Del Rey the results are somewhat 'inconclusive' due to potentially the smaller sample size of 40,000 and the fact that one of her generes is pop which is very general hence why Billie Eilish is being reccomeneded as one of the most popular pop artists.